In [1]:
import warnings
warnings.filterwarnings('always')
warnings.filterwarnings('ignore')
!pip install mlxtend
# data visualisation and manipulation
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import style
import seaborn as sns
import os
#configure
# sets matplotlib to inline and displays graphs below the corressponding cell.
%matplotlib inline
style.use('fivethirtyeight')
sns.set(style='whitegrid',color_codes=True)

#nltk
import nltk

#preprocessing
from nltk.corpus import stopwords  #stopwords
from nltk import word_tokenize,sent_tokenize # tokenizing
from nltk.stem import PorterStemmer,LancasterStemmer  # using the Porter Stemmer and Lancaster Stemmer and others
from nltk.stem.snowball import SnowballStemmer
from nltk.stem import WordNetLemmatizer  # lammatizer from WordNet

# for part-of-speech tagging
from nltk import pos_tag

# for named entity recognition (NER)
from nltk import ne_chunk

# vectorizers for creating the document-term-matrix (DTM)
from sklearn.feature_extraction.text import TfidfVectorizer,CountVectorizer

# BeautifulSoup libraray
from bs4 import BeautifulSoup 

import re # regex

#model_selection
from sklearn.model_selection import train_test_split,cross_validate
from sklearn.model_selection import KFold
from sklearn.model_selection import GridSearchCV

#evaluation
from sklearn.metrics import accuracy_score,roc_auc_score 
from sklearn.metrics import classification_report
from mlxtend.plotting import plot_confusion_matrix

#preprocessing scikit
from sklearn.preprocessing import MinMaxScaler,StandardScaler,LabelEncoder

#classifiaction.
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC,SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier,GradientBoostingClassifier,AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB,MultinomialNB
 
#stop-words
stop_words=set(nltk.corpus.stopwords.words('english'))

#keras
import keras
from keras.preprocessing.text import one_hot,Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense , Flatten ,Embedding,Input,CuDNNLSTM,LSTM
from keras.models import Model
from keras.preprocessing.text import text_to_word_sequence

#gensim w2v
#word2vec
from gensim.models import Word2Vec

import keras
from sklearn.model_selection import train_test_split,cross_validate
from keras.models import Sequential
from keras.layers import Dense , Flatten ,Embedding,Input,CuDNNLSTM,SpatialDropout1D,Bidirectional,LSTM
from keras.models import Model
from tensorflow.keras.utils import to_categorical
from keras.preprocessing.text import one_hot,Tokenizer
from keras.initializers import Constant
from keras.layers import ReLU
from keras.layers import Dropout
from keras.models import Sequential

from tensorflow import keras
from keras import optimizers
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from sklearn.metrics import f1_score, classification_report, log_loss
from tensorflow.keras.utils import to_categorical
from keras.callbacks import ModelCheckpoint, EarlyStopping

In [2]:
train = pd.read_csv('Train_Dataset.csv',index_col=False)
test = pd.read_csv('Test_Dataset.csv',index_col=False)
    
df=pd.concat([train,test])
# X_train, y_train = train["tweet"], train["sarcastic"]
# X_test, y_test = test["tweet"], test["sarcastic"]
# shuffling rows
df = df.sample(frac=1).reset_index(drop=True)
print(df.shape) 
df.head()

(8334, 2)


,tweet,sarcastic
0,I just wanna have the same love for Phil Famil...,0
1,Can't put into how am that is back,0
2,"colleges making fall announcments, remember yo...",0
3,im ollie im 5'8 and i think my 5'7 boyfriend i...,1
4,Birds are not real and you cannot tell me othe...,1


In [3]:
df['tweet'].isnull().sum()

0

In [4]:
df['sarcastic'].isnull().sum()

0

In [5]:
print(df.shape)
df.head()

(8334, 2)


,tweet,sarcastic
0,I just wanna have the same love for Phil Famil...,0
1,Can't put into how am that is back,0
2,"colleges making fall announcments, remember yo...",0
3,im ollie im 5'8 and i think my 5'7 boyfriend i...,1
4,Birds are not real and you cannot tell me othe...,1


In [6]:
# function to clean and pre-process the text.
from bs4 import BeautifulSoup
def clean_tweets(review):  
    
    # 1. Removing html tags
#     review_text = BeautifulSoup(review,"html5lib").get_text()
    
    # 2. Retaining only alphabets.
    review_text = re.sub("[^a-zA-Z]"," ",review)
    
    # 3. Converting to lower case and splitting
    word_tokens= review_text.lower().split()
    
    # 4. Remove stopwords
    le=WordNetLemmatizer()
    stop_words= set(stopwords.words("english"))     
    word_tokens= [le.lemmatize(w) for w in word_tokens if not w in stop_words]
    
    cleaned_review=" ".join(word_tokens)
    return cleaned_review

In [7]:
import gensim
# # load Google's pre-trained Word2Vec model.
pre_w2v_model = gensim.models.KeyedVectors.load_word2vec_format(r'GoogleNews-vectors-negative300.bin', binary=True) 

## if required to to train word2vec on our dataset

In [8]:
import nltk
import re
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.corpus import stopwords
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('omw-1.4')
tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')
sentences=[]
sum=0
for review in df['tweet']:
    sents=tokenizer.tokenize(review.strip())
    sum+=len(sents)
    for sent in sents:
        cleaned_sent=clean_tweets(sent)
        sentences.append(cleaned_sent.split()) # can use word_tokenize also.
print(sum)
print(len(sentences))

[nltk_data] Downloading package stopwords to /home/jovyan/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /home/jovyan/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /home/jovyan/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /home/jovyan/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


12368
12368


In [9]:
# total numberof extracted words.
vocab=pre_w2v_model.key_to_index
print("The total number of words are : ",len(vocab))

The total number of words are :  3000000


In [10]:
w2v_model=gensim.models.Word2Vec(sentences=sentences,vector_size=300,window=10,min_count=1)
w2v_model.train(sentences,epochs=10,total_examples=len(sentences))

(692617, 721780)

In [11]:
w2v_model.wv.get_vector('like')

array([ 1.49477348e-01,  6.25580549e-01,  2.05036968e-01,  2.90152878e-01,
        9.40137282e-02, -5.33790469e-01,  4.45721686e-01,  1.05016458e+00,
        4.05394472e-02,  2.13565417e-02, -4.84797917e-02, -5.52406430e-01,
        3.02482843e-02,  1.86381146e-01, -4.07398552e-01, -3.23596001e-01,
        6.97004646e-02,  8.45826492e-02,  1.96088731e-01, -1.26166552e-01,
       -5.24816692e-01, -5.46412952e-02,  3.62360507e-01,  6.29515052e-02,
        5.62209606e-01,  1.16894394e-01, -5.14419675e-01, -1.06687501e-01,
       -2.83807576e-01, -5.89534402e-01,  4.29628417e-02, -2.05852628e-01,
       -9.81995985e-02, -2.52909530e-02, -5.89904487e-02,  1.80491097e-02,
        1.97870076e-01, -6.28925383e-01,  2.61783116e-02, -2.25632131e-01,
       -3.50563169e-01,  5.72916679e-02,  9.62296501e-03, -2.15953887e-01,
        1.35499611e-01,  4.53073472e-01,  1.26402259e-01,  2.36439258e-01,
       -2.08284199e-01,  4.16210681e-01,  3.67933437e-02, -4.37828414e-02,
       -3.15222979e-01,  

In [12]:
# total numberof extracted words.
vocab=pre_w2v_model.key_to_index
print("The total number of words are : ",len(vocab))

The total number of words are :  3000000


In [13]:
pre_w2v_model.most_similar('like')

[('really', 0.575244665145874),
 ('weird', 0.5676319599151611),
 ('crazy', 0.5382446050643921),
 ('kind', 0.5310239195823669),
 ('maybe', 0.5220045447349548),
 ('loooove', 0.5187614560127258),
 ('anymore', 0.5177682042121887),
 ('Kinda_reminds', 0.5151873230934143),
 ('definitely', 0.5117844343185425),
 ('kinda_fishy', 0.5090124607086182)]

In [14]:
pre_w2v_model.similarity('good','like')

0.4638977

In [15]:
word_vec_dict={}
for word in vocab:
  word_vec_dict[word]=pre_w2v_model.get_vector(word)
print("The no of key-value pairs : ",len(word_vec_dict)) # should come equal to vocab size
  

The no of key-value pairs :  3000000


## Preparing data for keras embedding layer

In [16]:
df['tweet']=df['tweet'].apply(clean_tweets)

In [17]:
maxi=-1
for i,rev in enumerate(df['tweet']):
    tokens=rev.split()
    if(len(tokens)>maxi):
        maxi=len(tokens)
print(maxi)

63


In [18]:

tok = Tokenizer()
tok.fit_on_texts(df['tweet'])
vocab_size = len(tok.word_index) + 1
encd_rev = tok.texts_to_sequences(df['tweet'])

In [19]:
max_rev_len=63 # max lenght of a tweet
vocab_size = len(tok.word_index) + 1  # total no of words
embed_dim=300 # embedding dimension as choosen in word2vec constructor

In [20]:
# now padding to have a amximum length of 1565
from keras.preprocessing.sequence import pad_sequences
pad_rev= pad_sequences(encd_rev, maxlen=max_rev_len, padding='post')
pad_rev.shape   # note that we had 100K reviews and we have padded each review to have  a lenght of 1565 words.

(8334, 63)

## creating a w2v embedding matrix for input to embedding layer

In [21]:
# now creating the embedding matrix
embed_matrix=np.zeros(shape=(vocab_size,embed_dim))
for word,i in tok.word_index.items():
  embed_vector=word_vec_dict.get(word)
  if embed_vector is not None:  # word is in the vocabulary learned by the w2v model
    embed_matrix[i]=embed_vector
  # if word is not found then embed_vector corressponding to that vector will stay zero.

In [22]:
# prepare train and val sets first

Y=keras.utils.to_categorical(df['sarcastic'])  # one hot target as required by NN.
x_train,x_test,y_train,y_test=train_test_split(pad_rev,Y,test_size=0.20,random_state=42)

# #One Hot Encode Y values:
# encoder = LabelEncoder()
# y_train = encoder.fit_transform(train['sarcastic'].values)
# y_train = to_categorical(y_train) 

# y_test = encoder.fit_transform(test['sarcastic'].values)
# y_test = to_categorical(y_test) 

In [23]:

"""
Using a Bidiretional LSTM. 
"""
model = Sequential()
model.add(Embedding(input_dim = vocab_size, output_dim = embed_dim, input_length =max_rev_len))
model.add(SpatialDropout1D(0.3))
model.add(Bidirectional(LSTM(128, dropout=0.25, recurrent_dropout=0.25)))
model.add(Dense(64, activation='relu'))
#     model.add(Dropout(0.3))
model.add(Dense(2, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy',get_f1])
# return model
model.compile(optimizer=keras.optimizers.RMSprop(lr=1e-3),loss='binary_crossentropy',metrics=['accuracy'])
model.summary()

2023-01-10 16:55:18.363584: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-01-10 16:55:18.363784: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-01-10 16:55:18.367663: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-01-10 16:55:18.367817: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-01-10 16:55:18.367954: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from S

 compute capability: 8.6
2023-01-10 16:55:19.223209: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-01-10 16:55:19.223321: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1525] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 22290 MB memory:  -> device: 1, name: NVIDIA GeForce RTX 3090, pci bus id: 0000:09:00.0, compute capability: 8.6


NameError: name 'get_f1' is not defined

In [ ]:
# specify batch size and epocj=hs for training.
epochs=10
batch_size=64

In [ ]:
model.fit(x_train,y_train,epochs=epochs,batch_size=batch_size,validation_data=(x_test,y_test))

In [ ]:

predictions = model.predict(x_test, verbose=1)
print('Validation Loss:', log_loss(y_test, predictions))
print('Test Accuracy', (predictions.argmax(axis = 1) == y_test.argmax(axis = 1)).mean())
print('F1 Score:', f1_score(y_test.argmax(axis = 1), predictions.argmax(axis = 1), average='weighted'))


In [ ]:
checkpoint_path = "checkpoint/cp.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

# Create a callback that saves the model's weights
# cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
#                                                  save_weights_only=True,
#                                                 verbose=1)


cp_callback= [
                ModelCheckpoint(filepath=checkpoint_path, monitor= 'val_accuracy', verbose=1, save_best_only=True),
                EarlyStopping(patience = 2)
            ]
# Train the model with the new callback
model.fit(x_train, 
          y_train,  
          epochs=epochs,
          batch_size=batch_size,
          validation_data=(x_test,y_test),
          callbacks=cp_callback)  # Pass callback to training

# This may generate warnings related to saving the state of the optimizer.
# These warnings (and similar warnings throughout this notebook)
# are in place to discourage outdated usage, and can be ignored.

In [ ]:
def get_f1(y_true, y_pred): #taken from old keras source code
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    recall = true_positives / (possible_positives + K.epsilon())
    f1_val = 2*(precision*recall)/(precision+recall+K.epsilon())
    return f1_val

